In [ ]:
import os
import cv2
import random
import shutil
import time
import matplotlib
import glob
import operator
import psutil
import numpy as np
import math
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical

from shutil import copyfile
import pandas as pd
import PIL
from mlxtend.plotting import plot_confusion_matrix
from tqdm import tqdm 

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
tf.config.experimental.list_physical_devices()
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if physical_devices != []:
    print("Using GPU")
    for i in physical_devices:
        tf.config.experimental.set_memory_growth(i, True)
else:
    print("Using CPU")
    pass

In [ ]:
root_dir = str(input("Path where 'classify train' directory belongs: "))
classify_train = os.path.join(root_dir, 'classify train')

train_directory = os.path.join(classify_train, 'training')
validation_directory = os.path.join(classify_train, 'validation')
test_directory = os.path.join(classify_train, 'testing')

In [ ]:
trainX = []
valX = []
testX = []

trainY = []
valY = []
testY = []

for class_name in os.listdir(train_directory):
    class_dir = os.path.join(train_directory, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        x = cv2.imread(file_path)
        trainX.append(x)
        if 'normal' in file_path:
            trainY.append(0)
        elif 'pneumonia' in file_path:
            trainY.append(1)
        elif 'covid' in file_path:
            trainY.append(2)


for class_name in os.listdir(validation_directory):
    class_dir = os.path.join(validation_directory, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        x = cv2.imread(file_path)
        valX.append(x)  
        if 'normal' in file_path:
            valY.append(0)
        elif 'pneumonia' in file_path:
            valY.append(1)
        elif 'covid' in file_path:
            valY.append(2)

        
for class_name in os.listdir(test_directory):
    class_dir = os.path.join(test_directory, class_name)
    for file_name in os.listdir(class_dir):
        file_path = os.path.join(class_dir, file_name)
        x = cv2.imread(file_path)
        testX.append(x)  
        if 'normal' in file_path:
            testY.append(0)
        elif 'pneumonia' in file_path:
            testY.append(1)
        elif 'covid' in file_path:
            testY.append(2)

In [ ]:
trainX = np.array(trainX)
valX = np.array(valX)
testX = np.array(testX)

trainY = np.array(trainY)
valY = np.array(valY)
testY = np.array(testY)

In [ ]:
print(trainX.shape)
print(trainY.shape)

In [ ]:
print(valX.shape)
print(valY.shape)

In [ ]:
print(testX.shape)
print(testY.shape)

In [ ]:
trainX_norm = trainX.astype('float32')
valX_norm = valX.astype('float32')
testX_norm = testX.astype('float32')

trainX_norm = trainX_norm / 255.0
valX_norm = valX_norm / 255.0
testX_norm = testX_norm / 255.0

In [ ]:
print(trainX_norm.shape)
print(valX_norm.shape)
print(testX_norm.shape)

In [ ]:
trainY = to_categorical(trainY)
valY = to_categorical(valY)
testY = to_categorical(testY)

In [ ]:
x = np.concatenate((trainX_norm, testX_norm), axis=0)
y = np.concatenate((trainY, testY), axis=0)

In [ ]:
learning_rate = float(input("Enter the initial learning rate: "))
epoch = int(input("Enter the maximum number of epochs: "))
batch_size = int(input("Enter batch size: "))
lambd = float(input("Enter lambda for L2 regularization: "))

In [ ]:
char_name = str(input("Enter name of the characteristics folder: "))

char = os.path.join(root_dir, char_name)

if not os.path.exists(char):
    os.mkdir(char)
else:
    replace = str(input("Folder already exists ! Do you want to replace it ?(Y/N) "))
    if replace.upper() == 'Y':      
        shutil.rmtree(char)
        os.mkdir(char)
    elif replace.upper() == 'N':
        print('\nThe following folders already exist:')
        for i in os.listdir(root_dir): 
            print(i)
        char_name = str(input("\nEnter a new name of the characteristics folder: "))
        char = os.path.join(root_dir, char_name)
        if not os.path.exists(char):
            os.mkdir(char)
        else:
            print(f"{char_name} replaced")
            shutil.rmtree(char)
            os.mkdir(char)

In [ ]:
steps = 10 # change steps to 1 to apply exponential decay

def lr_schedule(epoch):
    return learning_rate * (0.1 ** int(epoch / steps))
    
best_model_address = os.path.join(char, 'best_model.h5')

In [ ]:
monitor = int(input("Press 1 to monitor Validation Accuracy\nPress 2 to monitor Validation Loss\nPress 3 to monitor Training Accuracy\nPress 4 to monitor Training Loss\n"))
patience = int(input('Enter number of epochs that will produce monitored quantity with no improvement after which training will be stopped: '))


if monitor == 1:
    metric = 'val_accuracy'
    mode = 'max'
    print("\nMONITORING VALIDATION ACCURACY..........\n")

elif monitor == 2:
    metric = 'val_loss'
    mode = 'min'
    print("\nMONITORING VALIDATION LOSS..........\n")

elif monitor == 3:
    metric = 'accuracy'
    mode = 'max'
    print("\nMONITORING TRAINING ACCURACY..........\n")

elif monitor == 4:
    metric = 'loss'
    mode = 'min'
    print("\nMONITORING TRAINING LOSS..........\n")

callback = [keras.callbacks.LearningRateScheduler(lr_schedule, verbose = 1),
            keras.callbacks.EarlyStopping(monitor = metric, min_delta = 0.001, patience = patience, verbose=1, mode = mode, restore_best_weights = True),
            keras.callbacks.ModelCheckpoint(best_model_address, monitor = metric, verbose=1, save_best_only=True, save_weights_only=False, mode = mode)]

print(f"\nTraining will stop if {metric} doesn't show any improvements for {patience} epcohs\n")

In [ ]:
no_layers = int(input('Conv2d with activation + Max-pool + Dropout for feature extraction = 1 feature extraction layer \nHow many of such feature extraction layers you want to use ? '))    
no_conv = int(input('How many conv2d layers you want to use in each feature extraction layer ? '))
no_filters = int(input('Put no. of filters in 1st conv2d layer: '))
size_filter = int(input('Enter size of filter (width or height): '))
f_dropout = int(input('Enter dropout rate for feature extraction: '))/100

no_d_layers = int(input('Dense with activation + Dropout for desnse layer = 1 dense layer \nHow many of such dense layers you want to use ? '))
d_neurons = int(input('Enter no.of neurons you want to use in 1st dense layer: '))
d_dropout = int(input('Enter dropout rate for dense layer: '))/100

In [ ]:
def Custom_Model():        
        
    model = Sequential(name = 'CUSTOM')
    
    
    # feature extraction
    m, n = 0, 0 # m = increamental factor of no. of filters, # n = total no. of filters in convolution layer
    for l in range(no_layers):
        m = 2**l  
        n = no_filters*m 
        for i in range(no_conv):
            model.add(Conv2D(n, 
                             (size_filter,size_filter), 
                             kernel_regularizer=l2(lambd), 
                             bias_regularizer=l2(lambd),
                             padding = 'same', 
                             input_shape = dim))
            model.add(LeakyReLU())
        model.add(MaxPooling2D(2, 2))
        model.add(Dropout(f_dropout))
    
    
    # flatten
    model.add(Flatten())
    
    
    # dense layer
    m, n = 0, 0
    for d in range(no_d_layers):
        m = 2**d
        n = d_neurons//m
        model.add(Dense(n, kernel_regularizer=l2(lambd), bias_regularizer=l2(lambd)))
        model.add(LeakyReLU())
        model.add(Dropout(d_dropout))
    model.add(Dense(output_neurons, output_activation))

    
    return model

In [ ]:
class_no = len(os.listdir(train_directory))

print("This is a " + str(class_no) + "-Class Classification")

if class_no <= 2:
    class_mode = 'binary'
    output_activation = 'sigmoid'
    output_neurons = 1
    losses = 'binary_crossentropy'

else:
    class_mode = 'categorical'
    output_activation = 'softmax'
    output_neurons = class_no
    losses = 'categorical_crossentropy'

In [ ]:
def optimizer_selection():
    print("\nSelect a optimizer which will reduce the loss of the model.\n")

    optimizer_select = int(input("Press 1 to select Stochastic Gradient Descent\nPress 2 to select RMSprop\nPress 3 to select Adagrad\nPress 4 to select Adadelta\nPress 5 to select Adam\nPress 6 to select Adamax\nPress 7 to select Nadam\n"))

    if optimizer_select == 1:
        optimizer = SGD(lr = learning_rate, decay = 1e-6, momentum = 0.9, nesterov = True)

    elif optimizer_select == 2:
        optimizer = RMSprop(learning_rate, rho = 0.9)

    elif optimizer_select == 3:
        optimizer = Adagrad(learning_rate)

    elif optimizer_select == 4:
        optimizer = Adadelta(learning_rate, rho = 0.95)

    elif optimizer_select == 5:
        optimizer = Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999, amsgrad = False)

    elif optimizer_select == 6:
        optimizer = Adamax(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999)

    elif optimizer_select == 7:
        optimizer = Nadam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.999)
   
    return optimizer

In [ ]:
h = int(input("Image Dimension(H or W): "))
w = h
color = int(input("Press 1 for RGB \nPress 2 for Grayscale"))
if color == 1:
    color_mode = 'rgb'
    dim = (h,w,3)
elif color == 2:
    color_mode = 'grayscale'
    dim = (h,w,1)

In [ ]:
optimizer = optimizer_selection()

In [ ]:
from sklearn.model_selection import KFold

n_fold = int(input("Enter number of folds: "))

train_scores, val_scores, test_scores, histories = list(), list(), list(), list()

kfold = KFold(n_fold, shuffle=True, random_state=1)

fold_no = 1

for train, val in kfold.split(x, y):
    model = Custom_Model()
    model.compile(optimizer = optimizer, loss = losses, metrics = ['accuracy', 
                                                               tf.keras.metrics.Precision(), 
                                                               tf.keras.metrics.Recall(), 
                                                               tf.keras.metrics.TruePositives(), 
                                                               tf.keras.metrics.TrueNegatives(), 
                                                               tf.keras.metrics.FalsePositives(),
                                                               tf.keras.metrics.FalseNegatives()])

    print('------------------------------------------------------------------------')
    print('Training for fold {}...'.format(fold_no))

    history = model.fit(x[train], y[train], 
                        epochs=epoch, 
                        batch_size=batch_size, 
                        validation_data=(x[val], y[val]), 
                        callbacks = callback, 
                        verbose=1)

    train_score = model.evaluate(x[train], y[train])[1]
    val_score = model.evaluate(x[val], y[val])[1]
    test_score = model.evaluate(testX_norm, testY)[1]

    print("\nTraining Acc: {},\nValidation Acc: {},\nTest Acc: {}".format(train_score, val_score, test_score))

    train_scores.append(train_score)
    val_scores.append(val_score)
    test_scores.append(test_score)
    histories.append(history)

    fold_no = fold_no + 1

In [ ]:
i = 0
val_acc = []
tr_acc = []
ts_acc = []

for i in range(len(val_scores)):
    val_acc.append(val_scores[i])
    tr_acc.append(train_scores[i])
    ts_acc.append(test_scores[i])

In [ ]:
for i in range(len(histories)):
    plt.title('Training Loss')
    plt.plot(histories[i].history['loss'], color='blue')

plt.show()

In [ ]:
for i in range(len(histories)):
    plt.title('Validation Loss')
    plt.plot(histories[i].history['val_loss'], color='orange')
    
plt.show()

In [ ]:
for i in range(len(histories)):
    plt.title('Training Accuracy')
    plt.plot(histories[i].history['accuracy'], color='blue')

plt.show()

In [ ]:
for i in range(len(histories)):
    plt.title('Validation Accuracy')
    plt.plot(histories[i].history['val_accuracy'], color='orange')

plt.show()

In [ ]:
cv_train = np.mean(tr_acc), np.std(tr_acc)
cv_train

In [ ]:
cv_val = np.mean(val_acc), np.std(val_acc)
cv_val

In [ ]:
cv_test = np.mean(ts_acc), np.std(ts_acc)
cv_test

In [ ]:
tr_loss = []
tr_acc = []
tr_val_loss = []
tr_val_acc = []

In [ ]:
i,j = 0,0
for i in range(len(histories[i].history['loss'])):
    tr_loss_epch = []
    tr_acc_epch = []
    tr_val_loss_epch = []
    tr_val_acc_epch = []
    for j in range(len(histories)):
        tr_loss_epch.append((histories[j].history['loss'][i]))
        tr_acc_epch.append((histories[j].history['accuracy'][i]))
        tr_val_loss_epch.append((histories[j].history['val_loss'][i]))
        tr_val_acc_epch.append((histories[j].history['val_accuracy'][i]))

    tr_loss.append(np.mean(tr_loss_epch))
    tr_acc.append(np.mean(tr_acc_epch))
    tr_val_loss.append(np.mean(tr_val_loss_epch))
    tr_val_acc.append(np.mean(tr_val_acc_epch))

In [ ]:
plt.plot(tr_loss)
plt.title('Cross Entropy Loss')
plt.savefig(os.path.join(char, 'avg_tr_loss.eps'))
plt.show()

In [ ]:
plt.plot(tr_val_loss)
plt.title('Validation Loss')
plt.savefig(os.path.join(char, 'avg_val_loss.eps'))
plt.show()

In [ ]:
plt.plot(tr_acc)
plt.title('Training Accuracy')
plt.savefig(os.path.join(char, 'avg_tr_acc.eps'))
plt.show()

In [ ]:
plt.plot(tr_val_acc)
plt.title('Validation Accuracy')
plt.savefig(os.path.join(char, 'avg_val_acc.eps'))
plt.show()

In [ ]:
plt.plot(tr_loss, 'r', label='Training Loss')
plt.plot(tr_val_loss, 'b', label='Validation Loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title('Training and validation Loss vs Epochs')
plt.legend()
accuracy_fig_name = "loss.eps"
plt.savefig(os.path.join(char, accuracy_fig_name))
plt.show()

In [ ]:
plt.plot(tr_acc, 'r', label='Training Accuracy')
plt.plot(tr_val_acc, 'b', label='Validation Accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title('Training and validation Accuracy vs Epochs')
plt.legend()
accuracy_fig_name = "acc.eps"
plt.savefig(os.path.join(char, accuracy_fig_name))
plt.show()

In [ ]:
from contextlib import redirect_stdout

readme_name_text = "readme.txt"
print(f"Please read the text file named {readme_name_text} for detailed information of the model.")

completeName_txt = os.path.join(char, readme_name_text) 

readme = open(completeName_txt, "w")

if len(os.listdir(train_directory)) > 2:
    readme.write(f"This is a {len(os.listdir(train_directory))}-class CLASSIFICATION")
else:
    readme.write("This is a BINARY CLASSIFICATION")


readme.write("\n\n--HYPERPARAMETERS--\n")
readme.write(f"\nInitial Learning Rate = {learning_rate}")
readme.write(f"\nNo. of epochs = {len(acc)}")
readme.write(f"\nBatch Size = {batch_size}")


readme.write("\n\n--MODEL-PARAMETERS--")
readme.write(f"\nDropout for feature extraction = {(int(f_dropout*100))} %")
readme.write(f"\nDropout for dense layer = {(int(d_dropout*100))} %")
readme.write(f"\nOptimizer = {optimizer}\n\n")


readme.write("Trained on a Custom Prebuilt Model\n")
readme.write(f"\nFilter size = {size_filter}x{size_filter}\n\n")
with redirect_stdout(readme):
    model.summary()
        
    
readme.write("\n\n--MODEL-PERFORMANCE--")
readme.write(f"\nTest Accuracy = {test_accuracy} %")
readme.write(f"\nTest Precision = {test_precision} %")
readme.write(f"\nTest Recall = {test_recall} %")
readme.write(f"\nTrue Positive = {tp}")
readme.write(f"\nTrue Negetive = {tn}")
readme.write(f"\nFalse Positive = {fp}")
readme.write(f"\nFalse Negetive = {fn}")
readme.write(f"\nSensitivity = {sensitivity}")
readme.write(f"\nSpecificity = {specificity}\n\n\n")


readme.write("\n\n--MODEL-CHARACTERISTICS--")
readme.write(f"\nacc = {acc}")
readme.write(f"\n\nval_acc = {val_acc}")
readme.write(f"\n\nloss = {loss}")
readme.write(f"\n\nval_loss = {val_loss}")


readme.write("\n\n--Classification Report--\n")
readme.write(classification_reports)

readme.write(f"\nSensitivity = {sensitivity*100} %")
readme.write(f"\nSpecificity = {specificity*100} %")


readme.write(f"\nExecution Time: {duration} seconds")

readme.write("\n\nCreated using Self-Regulated Image Classifier using Convolution Neural Network")

readme.close()